In [9]:
%load_ext autoreload
%autoreload 2
from catboost import CatBoostClassifier
from src.models.utils import read_feature_data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV, ParameterGrid
from sklearn.model_selection import train_test_split
from itertools import product, chain
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
os.chdir('..')

In [29]:
X, y, X_test=read_feature_data('less_50_btc_eth_2_week')
X_train = X.values
y_train = y.values

In [4]:
y.success.value_counts()

0    4075
1     682
Name: success, dtype: int64

In [10]:
def catboost_GridSearchCV(X, y, X_test, params, cat_features, n_splits=5):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, cat_features, n_splits=5)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

In [18]:
def cross_val(X, y, X_test, param, cat_features, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=500,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                eval_metric = 'Accuracy',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent'
        )
        
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        accuracy = accuracy_score(y_valid, y_pred)
        acc.append(accuracy)
    return sum(acc)/n_splits

In [21]:
params = {'depth':[2, 3, 4],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg':np.logspace(-20, -19, 3)
}

param = catboost_GridSearchCV(X_train, y_train, X_test, params, cat_features=[])




Params Tuning:: 100%|##########| 18/18 [01:50<00:00,  6.14s/it]

Acc: 0.8648302538681086
Params: {'depth': 4, 'l2_leaf_reg': 1e-20, 'loss_function': 'Logloss'}


In [26]:
clf = CatBoostClassifier(iterations=2500,
                        loss_function = param['loss_function'],
                        depth=param['depth'],
                        l2_leaf_reg = param['l2_leaf_reg'],
                        eval_metric = 'MCC',
                        leaf_estimation_iterations = 10,
                        use_best_model=True
)

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                    y_train, 
                                                    shuffle=True,
                                                    random_state=123,
                                                    train_size=0.8,
                                                    stratify=y_train
)
clf.fit(X_train, 
        y_train,
        cat_features=[],
        eval_set=(X_valid, y_valid)
)


0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 2.48ms	remaining: 6.19s
1:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 5.4ms	remaining: 6.75s
2:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 8.21ms	remaining: 6.83s
3:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 10.8ms	remaining: 6.76s
4:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 13.4ms	remaining: 6.71s
5:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 16.9ms	remaining: 7.01s
6:	learn: 0.0553141	test: 0.0000000	best: 0.0000000 (0)	total: 19.6ms	remaining: 6.97s
7:	learn: 0.0782461	test: 0.0000000	best: 0.0000000 (0)	total: 22.4ms	remaining: 6.96s
8:	learn: 0.0553141	test: 0.0000000	best: 0.0000000 (0)	total: 24.9ms	remaining: 6.88s
9:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 27.7ms	remaining: 6.89s
10:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 30.3ms	remaining: 6.86s
11:	learn: 0.0000000	test: 0.0000000	best: 

124:	learn: 0.3396171	test: 0.0425480	best: 0.1567340 (54)	total: 194ms	remaining: 3.68s
125:	learn: 0.3442660	test: 0.0425480	best: 0.1567340 (54)	total: 196ms	remaining: 3.69s
126:	learn: 0.3396171	test: 0.0425480	best: 0.1567340 (54)	total: 198ms	remaining: 3.69s
127:	learn: 0.3442660	test: 0.0425480	best: 0.1567340 (54)	total: 200ms	remaining: 3.7s
128:	learn: 0.3488577	test: 0.0425480	best: 0.1567340 (54)	total: 202ms	remaining: 3.71s
129:	learn: 0.3488577	test: 0.0425480	best: 0.1567340 (54)	total: 203ms	remaining: 3.71s
130:	learn: 0.3533945	test: 0.0425480	best: 0.1567340 (54)	total: 205ms	remaining: 3.71s
131:	learn: 0.3533945	test: 0.0425480	best: 0.1567340 (54)	total: 207ms	remaining: 3.71s
132:	learn: 0.3533945	test: 0.0425480	best: 0.1567340 (54)	total: 209ms	remaining: 3.71s
133:	learn: 0.3578785	test: 0.0425480	best: 0.1567340 (54)	total: 211ms	remaining: 3.72s
134:	learn: 0.3578785	test: 0.0425480	best: 0.1567340 (54)	total: 213ms	remaining: 3.74s
135:	learn: 0.3578785	

253:	learn: 0.5277223	test: 0.1324611	best: 0.1664785 (219)	total: 385ms	remaining: 3.4s
254:	learn: 0.5277223	test: 0.1324611	best: 0.1664785 (219)	total: 387ms	remaining: 3.4s
255:	learn: 0.5277223	test: 0.1324611	best: 0.1664785 (219)	total: 388ms	remaining: 3.4s
256:	learn: 0.5340407	test: 0.1324611	best: 0.1664785 (219)	total: 390ms	remaining: 3.4s
257:	learn: 0.5340407	test: 0.1324611	best: 0.1664785 (219)	total: 392ms	remaining: 3.41s
258:	learn: 0.5340407	test: 0.1324611	best: 0.1664785 (219)	total: 394ms	remaining: 3.4s
259:	learn: 0.5371770	test: 0.1324611	best: 0.1664785 (219)	total: 396ms	remaining: 3.41s
260:	learn: 0.5434054	test: 0.1324611	best: 0.1664785 (219)	total: 398ms	remaining: 3.41s
261:	learn: 0.5434054	test: 0.1324611	best: 0.1664785 (219)	total: 399ms	remaining: 3.41s
262:	learn: 0.5464979	test: 0.1324611	best: 0.1664785 (219)	total: 401ms	remaining: 3.41s
263:	learn: 0.5464979	test: 0.1324611	best: 0.1664785 (219)	total: 403ms	remaining: 3.41s
264:	learn: 0.5

385:	learn: 0.6843910	test: 0.1312380	best: 0.2233316 (328)	total: 577ms	remaining: 3.16s
386:	learn: 0.6869868	test: 0.1312380	best: 0.2233316 (328)	total: 579ms	remaining: 3.16s
387:	learn: 0.6895758	test: 0.1312380	best: 0.2233316 (328)	total: 588ms	remaining: 3.2s
388:	learn: 0.6895758	test: 0.1312380	best: 0.2233316 (328)	total: 589ms	remaining: 3.2s
389:	learn: 0.6895758	test: 0.1312380	best: 0.2233316 (328)	total: 591ms	remaining: 3.2s
390:	learn: 0.6921578	test: 0.1312380	best: 0.2233316 (328)	total: 593ms	remaining: 3.2s
391:	learn: 0.6921578	test: 0.1610738	best: 0.2233316 (328)	total: 595ms	remaining: 3.2s
392:	learn: 0.6921578	test: 0.1610738	best: 0.2233316 (328)	total: 597ms	remaining: 3.2s
393:	learn: 0.6921578	test: 0.1772848	best: 0.2233316 (328)	total: 599ms	remaining: 3.2s
394:	learn: 0.6947332	test: 0.1772848	best: 0.2233316 (328)	total: 601ms	remaining: 3.2s
395:	learn: 0.6947332	test: 0.1772848	best: 0.2233316 (328)	total: 603ms	remaining: 3.2s
396:	learn: 0.69473

507:	learn: 0.7668028	test: 0.2409669	best: 0.2589565 (428)	total: 770ms	remaining: 3.02s
508:	learn: 0.7668028	test: 0.2409669	best: 0.2589565 (428)	total: 772ms	remaining: 3.02s
509:	learn: 0.7692075	test: 0.2409669	best: 0.2589565 (428)	total: 774ms	remaining: 3.02s
510:	learn: 0.7692075	test: 0.2409669	best: 0.2589565 (428)	total: 775ms	remaining: 3.02s
511:	learn: 0.7716074	test: 0.2409669	best: 0.2589565 (428)	total: 777ms	remaining: 3.02s
512:	learn: 0.7740025	test: 0.2409669	best: 0.2589565 (428)	total: 778ms	remaining: 3.01s
513:	learn: 0.7740025	test: 0.2409669	best: 0.2589565 (428)	total: 780ms	remaining: 3.01s
514:	learn: 0.7763930	test: 0.2409669	best: 0.2589565 (428)	total: 781ms	remaining: 3.01s
515:	learn: 0.7763930	test: 0.2200035	best: 0.2589565 (428)	total: 782ms	remaining: 3.01s
516:	learn: 0.7763930	test: 0.2200035	best: 0.2589565 (428)	total: 784ms	remaining: 3s
517:	learn: 0.7763930	test: 0.2200035	best: 0.2589565 (428)	total: 785ms	remaining: 3s
518:	learn: 0.77

...

1962:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.06s	remaining: 836ms
1963:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.06s	remaining: 835ms
1964:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.06s	remaining: 833ms
1965:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.06s	remaining: 832ms
1966:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.06s	remaining: 830ms
1967:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.07s	remaining: 830ms
1968:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.07s	remaining: 829ms
1969:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.08s	remaining: 827ms
1970:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.08s	remaining: 826ms
1971:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.08s	remaining: 825ms
1972:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.08s	remaining: 823ms

2070:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.24s	remaining: 672ms
2071:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 670ms
2072:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 669ms
2073:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 667ms
2074:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 666ms
2075:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 665ms
2076:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.25s	remaining: 663ms
2077:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.26s	remaining: 662ms
2078:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.26s	remaining: 660ms
2079:	learn: 1.0000000	test: 0.1771892	best: 0.2589565 (428)	total: 3.26s	remaining: 659ms
2080:	learn: 1.0000000	test: 0.1771892	best: 0.2589565 (428)	total: 3.26s	remaining: 657ms

2180:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.43s	remaining: 502ms
2181:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.44s	remaining: 501ms
2182:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.44s	remaining: 499ms
2183:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.44s	remaining: 497ms
2184:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.44s	remaining: 497ms
2185:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.45s	remaining: 495ms
2186:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.45s	remaining: 493ms
2187:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.45s	remaining: 492ms
2188:	learn: 1.0000000	test: 0.2090781	best: 0.2589565 (428)	total: 3.45s	remaining: 490ms
2189:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.46s	remaining: 490ms
2190:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.46s	remaining: 488ms

2287:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.62s	remaining: 336ms
2288:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.63s	remaining: 334ms
2289:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.63s	remaining: 333ms
2290:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.63s	remaining: 331ms
2291:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.63s	remaining: 330ms
2292:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.64s	remaining: 328ms
2293:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.64s	remaining: 327ms
2294:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.64s	remaining: 325ms
2295:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.64s	remaining: 324ms
2296:	learn: 1.0000000	test: 0.1989446	best: 0.2589565 (428)	total: 3.64s	remaining: 322ms
2297:	learn: 1.0000000	test: 0.1871682	best: 0.2589565 (428)	total: 3.65s	remaining: 321ms

2400:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.81s	remaining: 157ms
2401:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.81s	remaining: 156ms
2402:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.82s	remaining: 154ms
2403:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.82s	remaining: 153ms
2404:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.82s	remaining: 151ms
2405:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.82s	remaining: 149ms
2406:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.83s	remaining: 148ms
2407:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.83s	remaining: 146ms
2408:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.83s	remaining: 145ms
2409:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.83s	remaining: 143ms
2410:	learn: 1.0000000	test: 0.1640946	best: 0.2589565 (428)	total: 3.83s	remaining: 141ms

In [31]:
clf = CatBoostClassifier(iterations=2500,
                        loss_function = param['loss_function'],
                        depth=param['depth'],
                        l2_leaf_reg = param['l2_leaf_reg'],
                        leaf_estimation_iterations = 10,)

clf.fit(X, y)

0:	learn: 0.6729373	total: 5.41ms	remaining: 13.5s
1:	learn: 0.6529361	total: 10.9ms	remaining: 13.6s
2:	learn: 0.6343728	total: 15.9ms	remaining: 13.2s
3:	learn: 0.6178466	total: 20.7ms	remaining: 12.9s
4:	learn: 0.6009204	total: 25.3ms	remaining: 12.6s
5:	learn: 0.5860679	total: 29.3ms	remaining: 12.2s
6:	learn: 0.5731241	total: 32.5ms	remaining: 11.6s
7:	learn: 0.5611156	total: 35.3ms	remaining: 11s
8:	learn: 0.5497237	total: 38ms	remaining: 10.5s
9:	learn: 0.5380737	total: 40.6ms	remaining: 10.1s
10:	learn: 0.5272477	total: 43.1ms	remaining: 9.76s
11:	learn: 0.5175730	total: 45.3ms	remaining: 9.4s
12:	learn: 0.5084949	total: 47.4ms	remaining: 9.07s
13:	learn: 0.4959559	total: 49.4ms	remaining: 8.77s
14:	learn: 0.4885180	total: 51.4ms	remaining: 8.51s
15:	learn: 0.4809780	total: 53.5ms	remaining: 8.3s
16:	learn: 0.4737783	total: 55.5ms	remaining: 8.11s
17:	learn: 0.4682785	total: 57.6ms	remaining: 7.94s
18:	learn: 0.4618913	total: 59.5ms	remaining: 7.77s
19:	learn: 0.4538175	total: 

172:	learn: 0.3230258	total: 405ms	remaining: 5.44s
173:	learn: 0.3228859	total: 407ms	remaining: 5.45s
174:	learn: 0.3224496	total: 410ms	remaining: 5.45s
175:	learn: 0.3223359	total: 414ms	remaining: 5.47s
176:	learn: 0.3221608	total: 417ms	remaining: 5.47s
177:	learn: 0.3217689	total: 420ms	remaining: 5.48s
178:	learn: 0.3213879	total: 422ms	remaining: 5.47s
179:	learn: 0.3209689	total: 424ms	remaining: 5.47s
180:	learn: 0.3206486	total: 428ms	remaining: 5.48s
181:	learn: 0.3199991	total: 430ms	remaining: 5.48s
182:	learn: 0.3197575	total: 432ms	remaining: 5.47s
183:	learn: 0.3195382	total: 435ms	remaining: 5.47s
184:	learn: 0.3193568	total: 437ms	remaining: 5.47s
185:	learn: 0.3190756	total: 439ms	remaining: 5.46s
186:	learn: 0.3187398	total: 442ms	remaining: 5.46s
187:	learn: 0.3185226	total: 444ms	remaining: 5.46s
188:	learn: 0.3181884	total: 446ms	remaining: 5.45s
189:	learn: 0.3179521	total: 448ms	remaining: 5.45s
190:	learn: 0.3175829	total: 450ms	remaining: 5.44s
191:	learn: 

351:	learn: 0.2787039	total: 795ms	remaining: 4.85s
352:	learn: 0.2785817	total: 798ms	remaining: 4.85s
353:	learn: 0.2784107	total: 800ms	remaining: 4.85s
354:	learn: 0.2782453	total: 802ms	remaining: 4.85s
355:	learn: 0.2781371	total: 804ms	remaining: 4.84s
356:	learn: 0.2780428	total: 807ms	remaining: 4.84s
357:	learn: 0.2778907	total: 814ms	remaining: 4.87s
358:	learn: 0.2777111	total: 817ms	remaining: 4.87s
359:	learn: 0.2771567	total: 819ms	remaining: 4.87s
360:	learn: 0.2769901	total: 821ms	remaining: 4.86s
361:	learn: 0.2765301	total: 826ms	remaining: 4.88s
362:	learn: 0.2760169	total: 829ms	remaining: 4.88s
363:	learn: 0.2758526	total: 831ms	remaining: 4.88s
364:	learn: 0.2756337	total: 833ms	remaining: 4.87s
365:	learn: 0.2755380	total: 835ms	remaining: 4.87s
366:	learn: 0.2752948	total: 837ms	remaining: 4.86s
367:	learn: 0.2750000	total: 839ms	remaining: 4.86s
368:	learn: 0.2745695	total: 842ms	remaining: 4.86s
369:	learn: 0.2744376	total: 844ms	remaining: 4.86s
370:	learn: 

593:	learn: 0.2323710	total: 1.38s	remaining: 4.42s
594:	learn: 0.2322065	total: 1.38s	remaining: 4.42s
595:	learn: 0.2320597	total: 1.38s	remaining: 4.42s
596:	learn: 0.2318415	total: 1.38s	remaining: 4.41s
597:	learn: 0.2316647	total: 1.39s	remaining: 4.41s
598:	learn: 0.2315669	total: 1.39s	remaining: 4.42s
599:	learn: 0.2314934	total: 1.4s	remaining: 4.42s
600:	learn: 0.2313566	total: 1.4s	remaining: 4.42s
601:	learn: 0.2308784	total: 1.4s	remaining: 4.41s
602:	learn: 0.2307813	total: 1.4s	remaining: 4.41s
603:	learn: 0.2306615	total: 1.4s	remaining: 4.41s
604:	learn: 0.2305651	total: 1.41s	remaining: 4.41s
605:	learn: 0.2304710	total: 1.41s	remaining: 4.4s
606:	learn: 0.2302323	total: 1.41s	remaining: 4.4s
607:	learn: 0.2300087	total: 1.41s	remaining: 4.4s
608:	learn: 0.2298424	total: 1.42s	remaining: 4.39s
609:	learn: 0.2297019	total: 1.42s	remaining: 4.39s
610:	learn: 0.2295791	total: 1.42s	remaining: 4.39s
611:	learn: 0.2294322	total: 1.42s	remaining: 4.39s
612:	learn: 0.229336

771:	learn: 0.2053247	total: 1.77s	remaining: 3.97s
772:	learn: 0.2051961	total: 1.78s	remaining: 3.97s
773:	learn: 0.2051058	total: 1.78s	remaining: 3.97s
774:	learn: 0.2049857	total: 1.78s	remaining: 3.96s
775:	learn: 0.2047595	total: 1.78s	remaining: 3.96s
776:	learn: 0.2045562	total: 1.78s	remaining: 3.96s
777:	learn: 0.2043739	total: 1.79s	remaining: 3.96s
778:	learn: 0.2042798	total: 1.79s	remaining: 3.95s
779:	learn: 0.2041295	total: 1.79s	remaining: 3.95s
780:	learn: 0.2040302	total: 1.79s	remaining: 3.95s
781:	learn: 0.2038859	total: 1.8s	remaining: 3.95s
782:	learn: 0.2037905	total: 1.8s	remaining: 3.94s
783:	learn: 0.2037046	total: 1.8s	remaining: 3.94s
784:	learn: 0.2035376	total: 1.8s	remaining: 3.94s
785:	learn: 0.2034060	total: 1.8s	remaining: 3.94s
786:	learn: 0.2033169	total: 1.81s	remaining: 3.93s
787:	learn: 0.2032447	total: 1.81s	remaining: 3.93s
788:	learn: 0.2029180	total: 1.81s	remaining: 3.93s
789:	learn: 0.2028475	total: 1.81s	remaining: 3.93s
790:	learn: 0.202

...

1637:	learn: 0.1198097	total: 3.73s	remaining: 1.96s
1638:	learn: 0.1197729	total: 3.74s	remaining: 1.96s
1639:	learn: 0.1197083	total: 3.74s	remaining: 1.96s
1640:	learn: 0.1196465	total: 3.74s	remaining: 1.96s
1641:	learn: 0.1195947	total: 3.75s	remaining: 1.96s
1642:	learn: 0.1195398	total: 3.75s	remaining: 1.95s
1643:	learn: 0.1194779	total: 3.75s	remaining: 1.95s
1644:	learn: 0.1194260	total: 3.75s	remaining: 1.95s
1645:	learn: 0.1193871	total: 3.75s	remaining: 1.95s
1646:	learn: 0.1193249	total: 3.76s	remaining: 1.95s
1647:	learn: 0.1192529	total: 3.76s	remaining: 1.94s
1648:	learn: 0.1191900	total: 3.76s	remaining: 1.94s
1649:	learn: 0.1191127	total: 3.76s	remaining: 1.94s
1650:	learn: 0.1190742	total: 3.76s	remaining: 1.94s
1651:	learn: 0.1190192	total: 3.77s	remaining: 1.93s
1652:	learn: 0.1189218	total: 3.77s	remaining: 1.93s
1653:	learn: 0.1188656	total: 3.77s	remaining: 1.93s
1654:	learn: 0.1187908	total: 3.77s	remaining: 1.93s
1655:	learn: 0.1187065	total: 3.78s	remaining:

1794:	learn: 0.1096804	total: 4.12s	remaining: 1.62s
1795:	learn: 0.1096137	total: 4.12s	remaining: 1.61s
1796:	learn: 0.1095436	total: 4.13s	remaining: 1.61s
1797:	learn: 0.1094681	total: 4.13s	remaining: 1.61s
1798:	learn: 0.1094060	total: 4.13s	remaining: 1.61s
1799:	learn: 0.1093713	total: 4.14s	remaining: 1.61s
1800:	learn: 0.1092738	total: 4.14s	remaining: 1.61s
1801:	learn: 0.1092135	total: 4.14s	remaining: 1.6s
1802:	learn: 0.1091620	total: 4.14s	remaining: 1.6s
1803:	learn: 0.1089766	total: 4.15s	remaining: 1.6s
1804:	learn: 0.1089211	total: 4.15s	remaining: 1.6s
1805:	learn: 0.1088760	total: 4.15s	remaining: 1.59s
1806:	learn: 0.1088380	total: 4.15s	remaining: 1.59s
1807:	learn: 0.1087874	total: 4.16s	remaining: 1.59s
1808:	learn: 0.1087599	total: 4.16s	remaining: 1.59s
1809:	learn: 0.1087254	total: 4.16s	remaining: 1.58s
1810:	learn: 0.1086387	total: 4.16s	remaining: 1.58s
1811:	learn: 0.1085902	total: 4.16s	remaining: 1.58s
1812:	learn: 0.1085359	total: 4.17s	remaining: 1.5

1959:	learn: 0.0987292	total: 4.5s	remaining: 1.24s
1960:	learn: 0.0986905	total: 4.51s	remaining: 1.24s
1961:	learn: 0.0986064	total: 4.51s	remaining: 1.24s
1962:	learn: 0.0985746	total: 4.52s	remaining: 1.24s
1963:	learn: 0.0985035	total: 4.52s	remaining: 1.23s
1964:	learn: 0.0984485	total: 4.52s	remaining: 1.23s
1965:	learn: 0.0984084	total: 4.52s	remaining: 1.23s
1966:	learn: 0.0983714	total: 4.52s	remaining: 1.23s
1967:	learn: 0.0983208	total: 4.53s	remaining: 1.22s
1968:	learn: 0.0982690	total: 4.53s	remaining: 1.22s
1969:	learn: 0.0982345	total: 4.53s	remaining: 1.22s
1970:	learn: 0.0981559	total: 4.53s	remaining: 1.22s
1971:	learn: 0.0981140	total: 4.54s	remaining: 1.21s
1972:	learn: 0.0980897	total: 4.54s	remaining: 1.21s
1973:	learn: 0.0980294	total: 4.54s	remaining: 1.21s
1974:	learn: 0.0979016	total: 4.54s	remaining: 1.21s
1975:	learn: 0.0978605	total: 4.54s	remaining: 1.2s
1976:	learn: 0.0978138	total: 4.54s	remaining: 1.2s
1977:	learn: 0.0977507	total: 4.55s	remaining: 1.

2117:	learn: 0.0898151	total: 4.9s	remaining: 884ms
2118:	learn: 0.0897651	total: 4.9s	remaining: 882ms
2119:	learn: 0.0897021	total: 4.91s	remaining: 879ms
2120:	learn: 0.0896203	total: 4.91s	remaining: 877ms
2121:	learn: 0.0895329	total: 4.91s	remaining: 875ms
2122:	learn: 0.0894714	total: 4.91s	remaining: 872ms
2123:	learn: 0.0894261	total: 4.91s	remaining: 870ms
2124:	learn: 0.0893562	total: 4.92s	remaining: 868ms
2125:	learn: 0.0893229	total: 4.92s	remaining: 866ms
2126:	learn: 0.0892934	total: 4.92s	remaining: 863ms
2127:	learn: 0.0892466	total: 4.92s	remaining: 861ms
2128:	learn: 0.0892191	total: 4.93s	remaining: 859ms
2129:	learn: 0.0891656	total: 4.93s	remaining: 856ms
2130:	learn: 0.0891207	total: 4.93s	remaining: 854ms
2131:	learn: 0.0890878	total: 4.93s	remaining: 852ms
2132:	learn: 0.0890262	total: 4.94s	remaining: 849ms
2133:	learn: 0.0889795	total: 4.94s	remaining: 847ms
2134:	learn: 0.0889458	total: 4.94s	remaining: 845ms
2135:	learn: 0.0889143	total: 4.94s	remaining: 8

2327:	learn: 0.0793538	total: 5.49s	remaining: 406ms
2328:	learn: 0.0793170	total: 5.49s	remaining: 403ms
2329:	learn: 0.0792783	total: 5.5s	remaining: 401ms
2330:	learn: 0.0792089	total: 5.5s	remaining: 399ms
2331:	learn: 0.0791571	total: 5.5s	remaining: 396ms
2332:	learn: 0.0791131	total: 5.5s	remaining: 394ms
2333:	learn: 0.0790704	total: 5.51s	remaining: 392ms
2334:	learn: 0.0789916	total: 5.52s	remaining: 390ms
2335:	learn: 0.0789384	total: 5.52s	remaining: 388ms
2336:	learn: 0.0788576	total: 5.52s	remaining: 385ms
2337:	learn: 0.0788334	total: 5.52s	remaining: 383ms
2338:	learn: 0.0787882	total: 5.53s	remaining: 380ms
2339:	learn: 0.0787407	total: 5.53s	remaining: 378ms
2340:	learn: 0.0787167	total: 5.53s	remaining: 376ms
2341:	learn: 0.0786593	total: 5.54s	remaining: 373ms
2342:	learn: 0.0785723	total: 5.54s	remaining: 371ms
2343:	learn: 0.0784919	total: 5.54s	remaining: 369ms
2344:	learn: 0.0784576	total: 5.54s	remaining: 366ms
2345:	learn: 0.0784373	total: 5.55s	remaining: 364

In [33]:
preds = clf.predict(X_test)
np.unique(preds,return_counts=True)

(array([0., 1.]), array([970,  31]))

In [47]:
submission = pd.DataFrame([X_test.OBS_ID, preds]).transpose()
submission.columns=['OBS_ID', 'success']

In [55]:
submission.success=submission.success.astype(int)

In [57]:
submission.dtypes

OBS_ID     float64
success      int64
dtype: object

In [58]:
submission.OBS_ID = submission.astype(int)

In [59]:
submission.dtypes

OBS_ID     int64
success    int64
dtype: object

In [60]:
submission

,OBS_ID,success
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
996,997,0
997,998,0
998,999,0
999,1000,0


In [61]:
submission.to_csv('data/submissions/submission_catboost_notebook.csv', index=None)

In [62]:
a = pd.read_csv('data/submissions/submission_catboost_notebook.csv')
a.head()

,OBS_ID,success
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [63]:
a.success.value_counts()

0    970
1     31
Name: success, dtype: int64

In [ ]:

sub = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':np.array(clf.predict(X_test)).astype(int)})
sub.to_csv('cat_sub_1.csv',index=False)